In [1]:
from sympy import *
import numpy as np
from scipy.spatial.transform import Rotation

In [2]:
# -- States --

# Quaternion
qw, qx, qy, qz = symbols(r"q_w, q_x, q_y, q_z")
quaternion = Matrix([qw, qx, qy, qz])

# Bias in gyro reading
bx_g, by_g, bz_g = symbols(r'b_{x_g}, b_{y_g}, b_{z_g}')
bias_gyro = Matrix([bx_g, by_g, bz_g]) 

# Bias in accelerometer reading
bx_a, by_a, bz_a = symbols(r'b_{x_a}, b_{y_a}, b_{z_a}')
bias_accelerometer = Matrix([bx_a, by_a, bz_a])

# Bias in magnetometer reading
bx_m, by_m, bz_m = symbols(r'b_{x_m}, b_{y_m}, b_{z_m}')
bias_magnetometer = Matrix([bx_m, by_m, bz_m])

states_s = Matrix([quaternion, bias_gyro, bias_magnetometer])

In [3]:
# -- Inputs --
wx, wy, wz = symbols(r"w_x, w_y, w_z")
gyro_reading = Matrix([wx, wy, wz])

inputs_u = [gyro_reading]

In [4]:
# -- Measurements --
ax, ay, az = symbols(r"a_x, a_y, a_z") # Accelerometer readings
accelerometer_reading = Matrix([ax, ay, az])

mag_x, mag_y, mag_z = symbols(r"m_x, m_y, m_z") # Magnetometer readings

In [5]:
# -- Parameters --
dt = Symbol(r"\Delta t") # Timestep, s

g = symbols("g")
gravity_vec = Matrix([0, 0, -g]) # Acceleration due to gravity in world frame NED, 3x m/s^2

nx, ny, nz = symbols(r"N_x, N_y, N_z")
mag_north = Matrix([nx, ny, nz]) # Local North vector, 3x Gauss

parameters = [dt, g, mag_north]

In [6]:
def rot_mat_from_quat(q):
    rotation_matrix = Matrix([
        [1 - 2*q[2]**2 - 2*q[3]**2, 2*q[1]*q[2] - 2*q[3]*q[0], 2*q[1]*q[3] + 2*q[2]*q[0]],
        [2*q[1]*q[2] + 2*q[3]*q[0], 1 - 2*q[1]**2 - 2*q[3]**2, 2*q[2]*q[3] - 2*q[1]*q[0]],
        [2*q[1]*q[3] - 2*q[2]*q[0], 2*q[2]*q[3] + 2*q[1]*q[0], 1 - 2*q[1]**2 - 2*q[2]**2]
    ])
    return rotation_matrix

In [7]:
def omega_skew_matrix(omega):
    w1, w2, w3 = omega
    mat = Matrix([
        [0, -w1, -w2, -w3],
        [w1, 0, w3, -w2],
        [w2, -w3, 0, w1],
        [w3, w2, -w1, 0]
    ])
    return mat

The state transition function ($f$) is defined as follows:
$$
s_{k+1} = f(s_k, u_k)
$$

In [8]:
state_transition_function_f = Matrix([
    quaternion + 1/2 * omega_skew_matrix(gyro_reading - bias_gyro) * quaternion * dt, 
    # world frame q_next
    bias_gyro, # assume bias evolves randomly
    bias_magnetometer, # assume bias evolves randomly
])

In [9]:
print(state_transition_function_f)

Matrix([[\Delta t*(q_x*(0.5*b_{x_g} - 0.5*w_x) + q_y*(0.5*b_{y_g} - 0.5*w_y) + q_z*(0.5*b_{z_g} - 0.5*w_z)) + q_w], [\Delta t*(q_w*(-0.5*b_{x_g} + 0.5*w_x) + q_y*(-0.5*b_{z_g} + 0.5*w_z) + q_z*(0.5*b_{y_g} - 0.5*w_y)) + q_x], [\Delta t*(q_w*(-0.5*b_{y_g} + 0.5*w_y) + q_x*(0.5*b_{z_g} - 0.5*w_z) + q_z*(-0.5*b_{x_g} + 0.5*w_x)) + q_y], [\Delta t*(q_w*(-0.5*b_{z_g} + 0.5*w_z) + q_x*(-0.5*b_{y_g} + 0.5*w_y) + q_y*(0.5*b_{x_g} - 0.5*w_x)) + q_z], [b_{x_g}], [b_{y_g}], [b_{z_g}], [b_{x_m}], [b_{y_m}], [b_{z_m}]])


In [10]:
state_transition_function_f

Matrix([
[  \Delta t*(q_x*(0.5*b_{x_g} - 0.5*w_x) + q_y*(0.5*b_{y_g} - 0.5*w_y) + q_z*(0.5*b_{z_g} - 0.5*w_z)) + q_w],
[\Delta t*(q_w*(-0.5*b_{x_g} + 0.5*w_x) + q_y*(-0.5*b_{z_g} + 0.5*w_z) + q_z*(0.5*b_{y_g} - 0.5*w_y)) + q_x],
[\Delta t*(q_w*(-0.5*b_{y_g} + 0.5*w_y) + q_x*(0.5*b_{z_g} - 0.5*w_z) + q_z*(-0.5*b_{x_g} + 0.5*w_x)) + q_y],
[\Delta t*(q_w*(-0.5*b_{z_g} + 0.5*w_z) + q_x*(-0.5*b_{y_g} + 0.5*w_y) + q_y*(0.5*b_{x_g} - 0.5*w_x)) + q_z],
[                                                                                                   b_{x_g}],
[                                                                                                   b_{y_g}],
[                                                                                                   b_{z_g}],
[                                                                                                   b_{x_m}],
[                                                                                                   b_{y_m}],
[

$$
s_{k+1} = As_k + Bu_k
$$

In [11]:
state_transition_matrix_A = state_transition_function_f.jacobian(states_s)
print(state_transition_matrix_A)

Matrix([[1, \Delta t*(0.5*b_{x_g} - 0.5*w_x), \Delta t*(0.5*b_{y_g} - 0.5*w_y), \Delta t*(0.5*b_{z_g} - 0.5*w_z), 0.5*\Delta t*q_x, 0.5*\Delta t*q_y, 0.5*\Delta t*q_z, 0, 0, 0], [\Delta t*(-0.5*b_{x_g} + 0.5*w_x), 1, \Delta t*(-0.5*b_{z_g} + 0.5*w_z), \Delta t*(0.5*b_{y_g} - 0.5*w_y), -0.5*\Delta t*q_w, 0.5*\Delta t*q_z, -0.5*\Delta t*q_y, 0, 0, 0], [\Delta t*(-0.5*b_{y_g} + 0.5*w_y), \Delta t*(0.5*b_{z_g} - 0.5*w_z), 1, \Delta t*(-0.5*b_{x_g} + 0.5*w_x), -0.5*\Delta t*q_z, -0.5*\Delta t*q_w, 0.5*\Delta t*q_x, 0, 0, 0], [\Delta t*(-0.5*b_{z_g} + 0.5*w_z), \Delta t*(-0.5*b_{y_g} + 0.5*w_y), \Delta t*(0.5*b_{x_g} - 0.5*w_x), 1, 0.5*\Delta t*q_y, -0.5*\Delta t*q_x, -0.5*\Delta t*q_w, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])


In [12]:
state_transition_matrix_A

Matrix([
[                                1,  \Delta t*(0.5*b_{x_g} - 0.5*w_x),  \Delta t*(0.5*b_{y_g} - 0.5*w_y),  \Delta t*(0.5*b_{z_g} - 0.5*w_z),  0.5*\Delta t*q_x,  0.5*\Delta t*q_y,  0.5*\Delta t*q_z, 0, 0, 0],
[\Delta t*(-0.5*b_{x_g} + 0.5*w_x),                                 1, \Delta t*(-0.5*b_{z_g} + 0.5*w_z),  \Delta t*(0.5*b_{y_g} - 0.5*w_y), -0.5*\Delta t*q_w,  0.5*\Delta t*q_z, -0.5*\Delta t*q_y, 0, 0, 0],
[\Delta t*(-0.5*b_{y_g} + 0.5*w_y),  \Delta t*(0.5*b_{z_g} - 0.5*w_z),                                 1, \Delta t*(-0.5*b_{x_g} + 0.5*w_x), -0.5*\Delta t*q_z, -0.5*\Delta t*q_w,  0.5*\Delta t*q_x, 0, 0, 0],
[\Delta t*(-0.5*b_{z_g} + 0.5*w_z), \Delta t*(-0.5*b_{y_g} + 0.5*w_y),  \Delta t*(0.5*b_{x_g} - 0.5*w_x),                                 1,  0.5*\Delta t*q_y, -0.5*\Delta t*q_x, -0.5*\Delta t*q_w, 0, 0, 0],
[                                0,                                 0,                                 0,                                 0,                 1,

In [13]:
input_matrix_B = state_transition_function_f.jacobian(inputs_u)

In [14]:
input_matrix_B

Matrix([
[-0.5*\Delta t*q_x, -0.5*\Delta t*q_y, -0.5*\Delta t*q_z],
[ 0.5*\Delta t*q_w, -0.5*\Delta t*q_z,  0.5*\Delta t*q_y],
[ 0.5*\Delta t*q_z,  0.5*\Delta t*q_w, -0.5*\Delta t*q_x],
[-0.5*\Delta t*q_y,  0.5*\Delta t*q_x,  0.5*\Delta t*q_w],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0]])

The observation model ($h$) is defined a follows:
$$
y_k = h(s_k, u_k)
$$

In [15]:
observation_model_h = Matrix([
    rot_mat_from_quat(quaternion).T * gravity_vec + bias_accelerometer, # accelerometer
    rot_mat_from_quat(quaternion).T * mag_north + bias_magnetometer, # magnetometer
])

In [16]:
observation_model_h

Matrix([
[                                                                 b_{x_a} - g*(-2*q_w*q_y + 2*q_x*q_z)],
[                                                                  b_{y_a} - g*(2*q_w*q_x + 2*q_y*q_z)],
[                                                               b_{z_a} - g*(-2*q_x**2 - 2*q_y**2 + 1)],
[N_x*(-2*q_y**2 - 2*q_z**2 + 1) + N_y*(2*q_w*q_z + 2*q_x*q_y) + N_z*(-2*q_w*q_y + 2*q_x*q_z) + b_{x_m}],
[N_x*(-2*q_w*q_z + 2*q_x*q_y) + N_y*(-2*q_x**2 - 2*q_z**2 + 1) + N_z*(2*q_w*q_x + 2*q_y*q_z) + b_{y_m}],
[N_x*(2*q_w*q_y + 2*q_x*q_z) + N_y*(-2*q_w*q_x + 2*q_y*q_z) + N_z*(-2*q_x**2 - 2*q_y**2 + 1) + b_{z_m}]])

In [17]:
observation_matrix_H = observation_model_h.jacobian(states_s)

In [18]:
observation_matrix_H

Matrix([
[               2*g*q_y,                          -2*g*q_z,                            2*g*q_w,                           -2*g*q_x, 0, 0, 0, 0, 0, 0],
[              -2*g*q_x,                          -2*g*q_w,                           -2*g*q_z,                           -2*g*q_y, 0, 0, 0, 0, 0, 0],
[                     0,                           4*g*q_x,                            4*g*q_y,                                  0, 0, 0, 0, 0, 0, 0],
[ 2*N_y*q_z - 2*N_z*q_y,             2*N_y*q_y + 2*N_z*q_z, -4*N_x*q_y + 2*N_y*q_x - 2*N_z*q_w, -4*N_x*q_z + 2*N_y*q_w + 2*N_z*q_x, 0, 0, 0, 1, 0, 0],
[-2*N_x*q_z + 2*N_z*q_x, 2*N_x*q_y - 4*N_y*q_x + 2*N_z*q_w,              2*N_x*q_x + 2*N_z*q_z, -2*N_x*q_w - 4*N_y*q_z + 2*N_z*q_y, 0, 0, 0, 0, 1, 0],
[ 2*N_x*q_y - 2*N_y*q_x, 2*N_x*q_z - 2*N_y*q_w - 4*N_z*q_x,  2*N_x*q_w + 2*N_y*q_z - 4*N_z*q_y,              2*N_x*q_x + 2*N_y*q_y, 0, 0, 0, 0, 0, 1]])

In [19]:
# Substitute equilibrium point values:
# Equilibrium quaternion (slight rotation)
# Zero biases and slight gyro
subs_dict = {
    # qw: 0.9862359, qx: 0.0544469, qy: 0.0806561, qz: 0.1336749,
    # qw: 1, qx: 0, qy: 0, qz: 0,  
    qw: 1, qx: 0, qy: 0.7071068, qz: 0.7071068,  
    bx_g: 0, by_g: 0, bz_g: 0,
    bx_a: 0, by_a: 0, bz_a: 0,
    bx_m: 0, by_m: 0, bz_m: 0,
    wx: 0.1, wy: 0.1, wz: 0.1,
    dt: 0.004,
    g: 9.81,
    mag_x: -0.01, mag_y: 0.2, mag_z: 0.4,
    nx: 0.2, ny: -0.01, nz: 0.4,
}

A_num = state_transition_matrix_A.subs(subs_dict).evalf()
B_num = input_matrix_B.subs(subs_dict).evalf()
H_num = observation_matrix_H.subs(subs_dict).evalf()

# Convert sympy matrices to numpy arrays for observability calculation
A_np = np.array(A_num.tolist(), dtype=np.float64)
H_np = np.array(H_num.tolist(), dtype=np.float64)

In [20]:
A_num

Matrix([
[   1.0, -0.0002, -0.0002, -0.0002,             0, 0.0014142136,  0.0014142136,   0,   0,   0],
[0.0002,     1.0,  0.0002, -0.0002,        -0.002, 0.0014142136, -0.0014142136,   0,   0,   0],
[0.0002, -0.0002,     1.0,  0.0002, -0.0014142136,       -0.002,             0,   0,   0,   0],
[0.0002,  0.0002, -0.0002,     1.0,  0.0014142136,            0,        -0.002,   0,   0,   0],
[     0,       0,       0,       0,           1.0,            0,             0,   0,   0,   0],
[     0,       0,       0,       0,             0,          1.0,             0,   0,   0,   0],
[     0,       0,       0,       0,             0,            0,           1.0,   0,   0,   0],
[     0,       0,       0,       0,             0,            0,             0, 1.0,   0,   0],
[     0,       0,       0,       0,             0,            0,             0,   0, 1.0,   0],
[     0,       0,       0,       0,             0,            0,             0,   0,   0, 1.0]])

In [21]:
B_num

Matrix([
[            0, -0.0014142136, -0.0014142136],
[        0.002, -0.0014142136,  0.0014142136],
[ 0.0014142136,         0.002,             0],
[-0.0014142136,             0,         0.002],
[            0,             0,             0],
[            0,             0,             0],
[            0,             0,             0],
[            0,             0,             0],
[            0,             0,             0],
[            0,             0,             0]])

In [22]:
H_num

Matrix([
[13.873435416, -13.873435416,         19.62,             0, 0, 0, 0,   0,   0,   0],
[           0,        -19.62, -13.873435416, -13.873435416, 0, 0, 0,   0,   0,   0],
[           0,             0,  27.746870832,             0, 0, 0, 0,   0,   0,   0],
[-0.579827576,   0.551543304,   -1.36568544,   -0.58568544, 0, 0, 0, 1.0,   0,   0],
[ -0.28284272,    1.08284272,    0.56568544,   0.193969712, 0, 0, 0,   0, 1.0,   0],
[  0.28284272,    0.30284272,  -0.745513016,  -0.014142136, 0, 0, 0,   0,   0, 1.0]])

In [23]:
# Build observability matrix [H; H A; H A^2; ... H A^(n-1)]
n_states = A_np.shape[0]
obs_matrix = H_np.copy()
for i in range(1, n_states):
    obs_matrix = np.vstack((obs_matrix, H_np @ np.linalg.matrix_power(A_np, i)))

rank_obs = np.linalg.matrix_rank(obs_matrix)

print(f"\nObservability matrix rank: {rank_obs} out of {n_states} states")

if rank_obs == n_states:
    print("The system is fully observable at the equilibrium point.")
else:
    print("The system is NOT fully observable at the equilibrium point.")


Observability matrix rank: 10 out of 10 states
The system is fully observable at the equilibrium point.


In [24]:
Matrix(obs_matrix.tolist())

Matrix([
[       13.873435416,       -13.873435416,              19.62,                   0,                     0,                     0,                     0,   0,   0,   0],
[                  0,              -19.62,      -13.873435416,       -13.873435416,                     0,                     0,                     0,   0,   0,   0],
[                  0,                   0,       27.746870832,                   0,                     0,                     0,                     0,   0,   0,   0],
[       -0.579827576,         0.551543304,        -1.36568544,         -0.58568544,                     0,                     0,                     0, 1.0,   0,   0],
[        -0.28284272,          1.08284272,         0.56568544,         0.193969712,                     0,                     0,                     0,   0, 1.0,   0],
[         0.28284272,          0.30284272,       -0.745513016,        -0.014142136,                     0,                     0,                 

In [25]:
from scipy.linalg import null_space

nullsp = null_space(obs_matrix)
if nullsp.shape[1] > 0:
    print("Basis vectors for unobservable states:")
    for i in range(nullsp.shape[1]):
        print(np.round(nullsp[:, i]))
        unobserved_states = [state for state, include in zip(states_s, abs(np.round(nullsp[:, i])) > 0.1) if include]
        print(f"Which corresponds to: {unobserved_states}")

In [26]:
process_noise_covariance_Q = np.diag([
    1.0e-6, # qw
    1.0e-6, # qx
    1.0e-6, # qy
    1.0e-6, # qz
    1.0e-6, # gyro bias (x)
    1.0e-6, # gyro bias (y)
    1.0e-6, # gyro bias (z)
    1.0e-6, # magnetometer bias (x)
    1.0e-6, # magnetometer bias (y)
    1.0e-6, # magnetometer bias (z)
])
measurement_noise_covariance_R = np.diag([
    1.0e2, # accelerometer (x)
    1.0e2, # accelerometer (y)
    1.0e2, # accelerometer (z)
    1.0e2, # magnetometer (x)
    1.0e2, # magnetometer (y)
    1.0e2, # magnetometer (z)
])

estimation_covariance_P = np.eye(n_states)
estimation_covariance_P = (
    A_np @ estimation_covariance_P @ A_np.T + process_noise_covariance_Q
)

kalman_gain_K = (
    estimation_covariance_P @ H_np.T @ np.linalg.inv(
        H_np @ estimation_covariance_P @ H_np.T + measurement_noise_covariance_R
    )
)
print(kalman_gain_K)

[[ 3.10919208e-02 -1.06881622e-02 -2.41193589e-02 -1.90596405e-05
   5.25354577e-06  2.91449935e-03]
 [-1.58000153e-02 -2.25088927e-02  2.70731178e-05  2.36488688e-03
   2.37383017e-03  1.66500951e-03]
 [ 6.28072619e-03 -4.42603105e-03  2.59563947e-02 -4.07077782e-04
   6.49767547e-04 -9.24988178e-04]
 [ 1.05024916e-02 -2.35285603e-02 -1.71426198e-02 -3.90850187e-03
  -1.97876962e-03 -9.86696211e-04]
 [ 3.75701661e-05  1.80025646e-05 -6.10048023e-05 -9.68144701e-06
  -8.46489615e-06 -3.41725620e-06]
 [ 9.06448568e-06 -3.80953171e-05 -8.59836434e-05  4.13161866e-06
   2.06497863e-06  8.32630413e-06]
 [ 4.53098173e-05  6.37735769e-05  1.37025688e-07  4.44555360e-06
   6.07860410e-07  3.74040384e-06]
 [ 4.10604352e-04 -1.34728485e-04  1.12951459e-04  9.85980697e-03
  -1.56216240e-05 -1.05896565e-05]
 [ 2.01118757e-04  2.81367518e-04 -1.80290342e-04 -1.56216240e-05
   9.87572521e-03 -2.61343803e-06]
 [ 8.13552241e-06  6.14586285e-05  2.56655531e-04 -1.05896565e-05
  -2.61343803e-06  9.8808